In [ ]:
!pip install -q torch
!pip install -q git+https://github.com/huggingface/transformers #huggingface transformers for downloading models weights
!pip install -q datasets #huggingface datasets to download and manipulate datasets
!pip install -q peft #Parameter efficient finetuning - for qLora Finetuning
!pip install -q bitsandbytes #For Model weights quantisation
!pip install -q trl #Transformer Reinforcement Learning - For Finetuning using Supervised Fine-tuning
!pip install -q wandb -U #Used to monitor the model score during training

In [1]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer # For supervised finetuning

c:\Users\dschr\anaconda3\envs\gpt-use-cases\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [ ]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

In [3]:
import pandas as pd

In [8]:
df = pd.read_csv('data_training_full.csv', header=0, index_col=0)
df.head()

,Argument ID,Conclusion,Stance,Premise,text,category,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,...,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
4568,A30388,We should adopt an austerity regime,against,we should not support the austerity regime bec...,we should not support the austerity regime bec...,"['Security: personal', 'Benevolence: caring', ...",0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
1142,A19198,We should legalize cannabis,in favor of,in many cases cannabis has wonderful health re...,in many cases cannabis has wonderful health re...,"['Self-direction: action', 'Security: personal...",0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
743,A18079,We should ban cosmetic surgery,in favor of,the long term effects of new procedures have n...,the long term effects of new procedures have n...,"['Security: personal', 'Benevolence: caring', ...",0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
4824,E01075,Devices with batteries should be designed so t...,in favor of,Having to replace an entire mobile phone (or t...,Having to replace an entire mobile phone (or t...,"['Security: personal', 'Universalism: nature']",0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2168,A22268,We should legalize cannabis,in favor of,we should legalize cannabis because it is harm...,we should legalize cannabis because it is harm...,"['Self-direction: action', 'Hedonism', 'Humili...",0,1,0,1,...,0,0,0,1,0,0,0,0,0,0


In [16]:
def convert_to_format(row):
    sentence1 = row['text']
    sentence2 = row['category']
    prompt = """Below is an instruction that describes a task paired with input that provides further context. Write a response that appropriately completes the request."""
    instruction = """Given the following input, your job is to label it with one or multiple underlying human values according to Schwartz Value Theory. The human values are implict present in the argument. \
        All available human values are: \
        "Self-direction: thought, Self-direction: action, Stimulation, Hedonism, Achievement, Power: dominance, Power: resources, Face, Security: personal, Security: societal, Tradition, Conformity: rules, Conformity: interpersonal, Humility, Benevolence: caring, Benevolence: dependability, Universalism: concern, Universalism: nature, Universalism: tolerance, Universalism: objectivity" . Use only these labels in your answer."""
    input = str(sentence1)
    response = f"""'human values': '{sentence2}'"""
    if len(input.strip()) == 0:  #  prompt + 2 new lines + ###instruction + new line + input + new line + ###response
        text = f"""<s>[INST]  {instruction} [/INST] \\n {response} </s>"""
    else:
        text = f"""<s>[INST]  {instruction} \n Input: {input} [/INST] \n {response} </s>"""

    
    # we need 4 columns for to train: instruction, input, output, text
    return pd.Series([instruction, input, response, text])

In [22]:
df = df.apply(convert_to_format, axis=1)
df.columns = ['instruction', 'input', 'output', 'text']

In [23]:
df.to_csv("data_instruct_format_mistral.csv", index=False)

In [25]:
print(df["text"][0])

<s>[INST]  Given the following input, your job is to label it with one or multiple underlying human values according to Schwartz Value Theory. The human values are implict present in the argument.         All available human values are:         "Self-direction: thought, Self-direction: action, Stimulation, Hedonism, Achievement, Power: dominance, Power: resources, Face, Security: personal, Security: societal, Tradition, Conformity: rules, Conformity: interpersonal, Humility, Benevolence: caring, Benevolence: dependability, Universalism: concern, Universalism: nature, Universalism: tolerance, Universalism: objectivity" . Use only these labels in your answer. 
 Input: we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same. in favor of We should ban human cloning [/INST] 
 'human values': '['Security: societal']' </s>


In [ ]:
from google.colab import drive

drive.mount('/content/drive')



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_instruct_format_mistral.csv')

In [21]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [26]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text', '__index_level_0__'],
    num_rows: 6876
})

In [19]:
# dataset = dataset.remove_columns("__index_level_0__")

In [ ]:
from huggingface_hub import login
login()

In [27]:
# The model that you want to train from the Hugging Face hub
model_name ="mistralai/Mistral-7B-Instruct-v0.1"
# model_name = "abhishek/llama-2-7b-hf-small-shards"

# Fine-tuned model name
new_model = "mistrala-7b-qlora-value-detector"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 1e-5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 50

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [28]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
print(compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

torch.float16


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=100, # the number of training steps the model will take
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",  # this is the text column in dataset 
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
# Train model
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
# Empty VRAM
import gc
del base_model
gc.collect()

del trainer
gc.collect()

In [ ]:
torch.cuda.empty_cache() # PyTorch thing

In [ ]:
gc.collect()

In [ ]:
from peft import get_peft_model
lora_config = LoraConfig.from_pretrained(new_model)
model = get_peft_model(model, lora_config)
model.push_to_hub(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
from random import randrange
sample = train_dataset [randrange(len(train_dataset ))]

prompt = f"""<s>
{sample['instruction']}
{sample['input']}
[INST]

"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)

print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"\nGround truth:\n{sample['output']}")

## Reload Model

In [ ]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
adapters_name = "danschr/llama-2-qlora-value-detector"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
dataset[1]["text"]

In [ ]:
device = "cuda:0"

inputs = tokenizer(input_sentence, return_tensors="pt").to(device)
outputs = m.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
pipe = pipeline(task="text-generation", model=m, tokenizer=tokenizer, max_length=300)
results = pipe(input_sentence)
print(results)

In [ ]:
!zip -r llama-2-qlora-value-detector.zip results llama-2-qlora-value-detector
!mv llama-2-qlora-value-detector.zip /content/drive/MyDrive

In [ ]:
!unzip /content/drive/MyDrive/llama-2-qlora-value-detector.zip -d .

In [ ]:
prompt_template = """### Instruction:
Given the following sentence, your job is to label it with one or multiple underlying human values according to Schwartz Value Theory. They human values are implict present in the argument. The answer should have the following format: Values: [value1, value2, ... ]. All available human values are: "Self-direction: thought, Self-direction: action, Stimulation, Hedonism, Achievement, Power: dominance, Power: resources, Face, Security: personal, Security: societal, Tradition, Conformity: rules, Conformity: interpersonal, Humility, Benevolence: caring, Benevolence: dependability, Universalism: concern, Universalism: nature, Universalism: tolerance, Universalism: objectivity" . Use only these labels in your answer.
### Input:
{}

### Response:
"""

sentence = dataset[10]["input"]
input_sentence = prompt_template.format(sentence.strip())
print(input_sentence)

In [ ]:
import re
import json

def format_results(s):
  pattern = r'```json\n(.*?)\n```'

  # Find all occurrences of JSON objects in the string
  json_matches = re.findall(pattern, s, re.DOTALL)
  if not json_matches:
    # try to find 2nd pattern
    pattern = r'\{.*?"sentence":.*?"negation":.*?\}'
    json_matches = re.findall(pattern, s)

  # Return the first JSON object found, or None if no match is found
  return json.loads(json_matches[0]) if json_matches else None